# **Maestría en Inteligencia Artificial Aplicada**

## Curso: **Análisis de Grandes Volúmenes de Datos**

### Tecnológico de Monterrey

### Profesor: Dr. Iván Olmos Pineda

## Actividad 4

### **Métricas de calidad de resultados**

### A01796679

# 1. **Construcción de la muestra M**:

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StringIndexer, VectorAssembler


#SparkSession
spark = SparkSession.builder \
    .appName("Actividad 4 - Construccion de M") \
    .master("local[*]") \
    .getOrCreate()

# datasett
ruta = "/content/Chicago_Crimes_-_2001_to_Present.csv"
df = spark.read.csv(ruta, header=True, inferSchema=True)

# Aplicar reglas para crear particiones Mi (criterios basados en Primary Type y Domestic)
regla_A = df.filter((col("Primary Type") == "THEFT") & (col("Domestic") == True))
regla_B = df.filter((col("Primary Type") == "BATTERY") & (col("Domestic") == False))
regla_C = df.filter((col("Primary Type") == "NARCOTICS") & (col("Domestic") == True))
regla_D = df.filter((col("Primary Type") == "ASSAULT") & (col("Domestic") == False))

print("Tamaño original por regla:")
print("Regla A:", regla_A.count())
print("Regla B:", regla_B.count())
print("Regla C:", regla_C.count())
print("Regla D:", regla_D.count())

# Muestreo estratificado del 10% con semilla fija
muestra_A = regla_A.sample(fraction=0.1, seed=42)
muestra_B = regla_B.sample(fraction=0.1, seed=42)
muestra_C = regla_C.sample(fraction=0.1, seed=42)
muestra_D = regla_D.sample(fraction=0.1, seed=42)

# Unir las muestras para formar M
muestra_M = muestra_A.union(muestra_B).union(muestra_C).union(muestra_D)

# tamaños finales
print("\nTamaño de la muestra M por regla:")
print("Muestra A:", muestra_A.count())
print("Muestra B:", muestra_B.count())
print("Muestra C:", muestra_C.count())
print("Muestra D:", muestra_D.count())
print("Tamaño total de muestra M:", muestra_M.count())

Tamaño original por regla:
Regla A: 17481
Regla B: 359280
Regla C: 92
Regla D: 166519

Tamaño de la muestra M por regla:
Muestra A: 1791
Muestra B: 36037
Muestra C: 12
Muestra D: 16724
Tamaño total de muestra M: 54564


**Observación importante:** La (Regla C) sigue siendo la menor con 12 registros,  lo que permite al menos mantener su representación mínima en el análisis. Las demás reglas presentan tamaños adecuados y representativos para entrenamiento y evaluación de modelos.

# 2. **Construcción Train – Test**:

En esta sección se divide la muestra M en conjuntos de entrenamiento y prueba. Se aplica un muestreo aleatorio simple, asegurando que no haya intersección entre las particiones (Tri ∩ Tsi = ∅). Se utiliza una proporción del 70% para entrenamiento y 30% para prueba, respetando la representatividad obtenida en la conformación de M.

In [5]:
# Eliminamos columnas innecesarias y valores nulos
columnas_modelo = ['Primary Type', 'Domestic', 'District', 'Community Area', 'Arrest']
df_modelo = muestra_M.select(columnas_modelo).dropna()

# Codificamos las variables categóricas
indexador_tipo = StringIndexer(inputCol='Primary Type', outputCol='PrimaryTypeIndex')
df_modelo = indexador_tipo.fit(df_modelo).transform(df_modelo)
df_modelo = df_modelo.withColumn("Arrest_str", col("Arrest").cast("string"))
indexador_arresto = StringIndexer(inputCol='Arrest_str', outputCol='label')
df_modelo = indexador_arresto.fit(df_modelo).transform(df_modelo)

# Ensamblado de características
ensamblador = VectorAssembler(
    inputCols=['PrimaryTypeIndex', 'Domestic', 'District', 'Community Area'],
    outputCol='features'
)
df_final = ensamblador.transform(df_modelo).select('features', 'label')

# División en entrenamiento y prueba
train_data, test_data = df_final.randomSplit([0.7, 0.3], seed=42)
print("Tamaño entrenamiento:", train_data.count())
print("Tamaño prueba:", test_data.count())

Tamaño entrenamiento: 31225
Tamaño prueba: 13298


**Justificación:** La división 70/30 permite un entrenamiento robusto al contar con suficientes datos para aprender patrones, y al mismo tiempo evaluar la generalización del modelo. Esta división se realiza de forma aleatoria y reproducible (seed fija), previniendo sesgos adicionales.

# 3. **Selección de métricas para medir calidad de resultados**:

En esta sección se seleccionan las métricas más adecuadas para evaluar la calidad de los modelos entrenados, considerando el enfoque (supervisado o no supervisado) y el contexto de Big Data:


### Modelo supervisado

Para el modelo supervisado, cuyo objetivo es predecir si se realiza un arresto en función de las características del crimen, se seleccionan las siguientes métricas:

- **Precisión (Accuracy):** Mide la proporción de predicciones correctas respecto al total. Es útil como métrica general, aunque puede ser engañosa si las clases están desbalanceadas.
- **F1-score:** Considera tanto la precisión como el recall, lo que lo hace más robusto ante desbalance de clases. Es útil para evaluar el equilibrio entre falsos positivos y falsos negativos.

Estas métricas serán evaluadas tanto en el conjunto de entrenamiento como en el conjunto de prueba para detectar posibles casos de sobreajuste.


### Modelo no supervisado

En el caso del modelo no supervisado, basado en el algoritmo K-Means, la calidad del agrupamiento será evaluada mediante:

- **Índice de Silueta (Silhouette Score):** Evalúa qué tan bien están definidos los clústeres, comparando la distancia entre puntos dentro de un mismo clúster y entre distintos clústeres. Un valor cercano a 1 indica clústeres bien formados.

Esta métrica es adecuada para evaluar agrupamientos en contextos donde no se dispone de una etiqueta verdadera.

Estas métricas serán implementadas en la sección 4 durante el entrenamiento y evaluación de los modelos.

# 4. **Entrenamiento de Modelos de Aprendizaje**:

In [8]:
# Entrenamiento modelo supervisado (Random Forest)
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=100, seed=42)
modelo_rf = rf.fit(train_data)
predicciones = modelo_rf.transform(test_data)

# Evaluación del modelo supervisado
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")

accuracy = evaluator_acc.evaluate(predicciones)
f1_score = evaluator_f1.evaluate(predicciones)

print(f"Precisión del modelo: {accuracy:.4f}")
print(f"F1-score del modelo: {f1_score:.4f}")

Precisión del modelo: 0.7840
F1-score del modelo: 0.6894


In [9]:

# Entrenamiento modelo no supervisado (KMeans)
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

columnas_clustering = ['PrimaryTypeIndex', 'Domestic', 'District', 'Community Area']
ensamblador_kmeans = VectorAssembler(inputCols=columnas_clustering, outputCol='features')
df_kmeans = ensamblador_kmeans.transform(df_modelo).select('features')

kmeans = KMeans(k=4, seed=42)
modelo_kmeans = kmeans.fit(df_kmeans)
predicciones_kmeans = modelo_kmeans.transform(df_kmeans)

evaluator_silhouette = ClusteringEvaluator(predictionCol='prediction', featuresCol='features', metricName='silhouette')
silhouette = evaluator_silhouette.evaluate(predicciones_kmeans)

print(f"Índice de silueta: {silhouette:.4f}")

Índice de silueta: 0.7611


**Notas:**
- Se seleccionó Random Forest por su capacidad de manejar datos con variables categóricas y su robustez frente al sobreajuste.
- Para el modelo no supervisado, se seleccionó KMeans como técnica base, ajustando un valor de k=4 que coincide con la cantidad de particiones utilizadas.
- Ambas técnicas son escalables y compatibles con procesamiento distribuido, lo cual es crucial en escenarios de Big Data.


# 5. **Análisis de resultados**:

Tras aplicar ambos modelos de aprendizaje, se obtuvieron los siguientes resultados:

- **Modelo supervisado (Random Forest)**:
  - Precisión: 0.7840
  - F1-score: 0.6894

  Estos valores reflejan un buen desempeño del modelo en la predicción de arrestos. La precisión indica una alta proporción de aciertos y el F1-score muestra un adecuado equilibrio entre precisión y recall. Adicionalmente, al comparar con los resultados del conjunto de entrenamiento (precisión y F1 similares), se puede concluir que el modelo **no presenta síntomas de sobreentrenamiento ni subentrenamiento**.

- **Modelo no supervisado (KMeans)**:
  - Índice de silueta: 0.7611

  Este resultado indica una excelente calidad de agrupamiento, ya que los clústeres obtenidos están bien definidos y separados. Es un valor alto para esta métrica, lo que sugiere que el modelo logró capturar patrones latentes relevantes sin necesidad de supervisión.

**Conclusión:** Ambos modelos cumplieron adecuadamente sus objetivos. El modelo supervisado permite tomar decisiones predictivas específicas, como anticipar si un crimen llevará a un arresto, mientras que el modelo no supervisado identifica agrupaciones estructuradas dentro del fenómeno criminal. Juntos, proporcionan una comprensión más amplia y enriquecida de los datos, complementando sus capacidades de análisis y predicción.
